In [ ]:
import sys
import os
import importlib
import multiprocessing as mp

from tqdm import tqdm
import numpy as np
import pandas as pd
import glob
import torch
from copy import copy

from torch.utils.data import DataLoader

import pandas as pd
import timm
from torch import nn
import torch
import torchaudio as ta
from torch.cuda.amp import autocast
import random

from torch.nn import functional as F
from torch.distributions import Beta
from torch.nn.parameter import Parameter
from torch.utils.data import Dataset

import numpy as np
import librosa
import ast

import os
from types import SimpleNamespace
import numpy as np

import numpy as np
import pandas as pd
import importlib
import sys
import random
from tqdm import tqdm
import gc
import argparse
import torch
from torch import optim
from torch.cuda.amp import GradScaler, autocast
from collections import defaultdict
import cv2
from copy import copy
import os
from transformers import get_cosine_schedule_with_warmup
from torch.utils.data import SequentialSampler, DataLoader
from pathlib import Path

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
def set_seed(seed=1234):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.benchmark = True

# Config

In [ ]:
cfg = SimpleNamespace()

# paths
cfg.data_folder = ''
cfg.name = "julian"
cfg.data_dir = "../input/birdclef-2023/"
cfg.train_data_folder = cfg.data_dir + "train_audio/"
cfg.val_data_folder = cfg.data_dir + "train_audio/"
cfg.output_dir = "first_model"

# dataset
cfg.dataset = "base_ds"
cfg.min_rating = 0
cfg.val_df = None
cfg.batch_size_val = 1
cfg.train_aug = None
cfg.val_aug = None
cfg.test_augs = None
cfg.wav_len_val = 5  # seconds
cfg.min_rating = 2.0
cfg.wav_crop_len = 30  # seconds
# audio
cfg.img_height = 256
cfg.img_weight = 512
cfg.window_size = 2048
cfg.sample_rate = 32000
cfg.fmin = 16
cfg.fmax = 16386
cfg.power = 2
cfg.mel_bins = cfg.img_height
cfg.hop_size = 512 #int(cfg.sample_rate * cfg.wav_crop_len / (cfg.img_weight - 1))
cfg.top_db = 80.0

# img model
cfg.backbone = "resnet18"
cfg.pretrained = True
cfg.pretrained_weights = None
cfg.train = True
cfg.val = False
cfg.in_chans = 1

cfg.alpha = 1
cfg.eval_epochs = 1
cfg.eval_train_epochs = 1
cfg.warmup = 0

cfg.mel_norm = False

cfg.label_smoothing = 0

cfg.remove_pretrained = []

# training
cfg.seed = 123
cfg.save_val_data = True

# ressources
cfg.mixed_precision = True
cfg.gpu = 0
cfg.num_workers = 4 # 18
cfg.drop_last = True

cfg.mixup2 = 0

cfg.label_smoothing = 0

cfg.mixup_2x = False


cfg.birds = np.array(['abethr1', 'abhori1', 'abythr1', 'afbfly1', 'afdfly1', 'afecuc1',
       'affeag1', 'afgfly1', 'afghor1', 'afmdov1', 'afpfly1', 'afpkin1',
       'afpwag1', 'afrgos1', 'afrgrp1', 'afrjac1', 'afrthr1', 'amesun2',
       'augbuz1', 'bagwea1', 'barswa', 'bawhor2', 'bawman1', 'bcbeat1',
       'beasun2', 'bkctch1', 'bkfruw1', 'blacra1', 'blacuc1', 'blakit1',
       'blaplo1', 'blbpuf2', 'blcapa2', 'blfbus1', 'blhgon1', 'blhher1',
       'blksaw1', 'blnmou1', 'blnwea1', 'bltapa1', 'bltbar1', 'bltori1',
       'blwlap1', 'brcale1', 'brcsta1', 'brctch1', 'brcwea1', 'brican1',
       'brobab1', 'broman1', 'brosun1', 'brrwhe3', 'brtcha1', 'brubru1',
       'brwwar1', 'bswdov1', 'btweye2', 'bubwar2', 'butapa1', 'cabgre1',
       'carcha1', 'carwoo1', 'categr', 'ccbeat1', 'chespa1', 'chewea1',
       'chibat1', 'chtapa3', 'chucis1', 'cibwar1', 'cohmar1', 'colsun2',
       'combul2', 'combuz1', 'comsan', 'crefra2', 'crheag1', 'crohor1',
       'darbar1', 'darter3', 'didcuc1', 'dotbar1', 'dutdov1', 'easmog1',
       'eaywag1', 'edcsun3', 'egygoo', 'equaka1', 'eswdov1', 'eubeat1',
       'fatrav1', 'fatwid1', 'fislov1', 'fotdro5', 'gabgos2', 'gargan',
       'gbesta1', 'gnbcam2', 'gnhsun1', 'gobbun1', 'gobsta5', 'gobwea1',
       'golher1', 'grbcam1', 'grccra1', 'grecor', 'greegr', 'grewoo2',
       'grwpyt1', 'gryapa1', 'grywrw1', 'gybfis1', 'gycwar3', 'gyhbus1',
       'gyhkin1', 'gyhneg1', 'gyhspa1', 'gytbar1', 'hadibi1', 'hamerk1',
       'hartur1', 'helgui', 'hipbab1', 'hoopoe', 'huncis1', 'hunsun2',
       'joygre1', 'kerspa2', 'klacuc1', 'kvbsun1', 'laudov1', 'lawgol',
       'lesmaw1', 'lessts1', 'libeat1', 'litegr', 'litswi1', 'litwea1',
       'loceag1', 'lotcor1', 'lotlap1', 'luebus1', 'mabeat1', 'macshr1',
       'malkin1', 'marsto1', 'marsun2', 'mcptit1', 'meypar1', 'moccha1',
       'mouwag1', 'ndcsun2', 'nobfly1', 'norbro1', 'norcro1', 'norfis1',
       'norpuf1', 'nubwoo1', 'pabspa1', 'palfly2', 'palpri1', 'piecro1',
       'piekin1', 'pitwhy', 'purgre2', 'pygbat1', 'quailf1', 'ratcis1',
       'raybar1', 'rbsrob1', 'rebfir2', 'rebhor1', 'reboxp1', 'reccor',
       'reccuc1', 'reedov1', 'refbar2', 'refcro1', 'reftin1', 'refwar2',
       'rehblu1', 'rehwea1', 'reisee2', 'rerswa1', 'rewsta1', 'rindov',
       'rocmar2', 'rostur1', 'ruegls1', 'rufcha2', 'sacibi2', 'sccsun2',
       'scrcha1', 'scthon1', 'shesta1', 'sichor1', 'sincis1', 'slbgre1',
       'slcbou1', 'sltnig1', 'sobfly1', 'somgre1', 'somtit4', 'soucit1',
       'soufis1', 'spemou2', 'spepig1', 'spewea1', 'spfbar1', 'spfwea1',
       'spmthr1', 'spwlap1', 'squher1', 'strher', 'strsee1', 'stusta1',
       'subbus1', 'supsta1', 'tacsun1', 'tafpri1', 'tamdov1', 'thrnig1',
       'trobou1', 'varsun2', 'vibsta2', 'vilwea1', 'vimwea1', 'walsta1',
       'wbgbir1', 'wbrcha2', 'wbswea1', 'wfbeat1', 'whbcan1', 'whbcou1',
       'whbcro2', 'whbtit5', 'whbwea1', 'whbwhe3', 'whcpri2', 'whctur2',
       'wheslf1', 'whhsaw1', 'whihel1', 'whrshr1', 'witswa1', 'wlwwar',
       'wookin1', 'woosan', 'wtbeat1', 'yebapa1', 'yebbar1', 'yebduc1',
       'yebere1', 'yebgre1', 'yebsto1', 'yeccan1', 'yefcan', 'yelbis1',
       'yenspu1', 'yertin1', 'yesbar1', 'yespet1', 'yetgre1', 'yewgre1'])


cfg.n_classes = len(cfg.birds)


# training
cfg.lr = 0.0001
cfg.epochs = 20
cfg.batch_size = 64
cfg.batch_size_val = 64
cfg.backbone = "resnet34"


cfg.save_val_data = True
cfg.mixed_precision = True

cfg.mixup = True
cfg.mix_beta = 1


cfg.train_df1 = "../input/birdclef-2023/train_metadata.csv"
cfg.train_df2 = "../input/birdclef-22-duration/train_metadata_with_duration.csv"


cfg.device = 'cuda' if torch.cuda.is_available() else 'cpu'

cfg.tr_collate_fn = None
cfg.val_collate_fn = None
cfg.val = False

cfg.dev = False
cfg.model = 'RN34'

cfg

namespace(data_folder='',
          name='julian',
          data_dir='../input/birdclef-2023/',
          train_data_folder='../input/birdclef-2023/train_audio/',
          val_data_folder='../input/birdclef-2023/train_audio/',
          output_dir='first_model',
          dataset='base_ds',
          min_rating=2.0,
          val_df=None,
          batch_size_val=64,
          train_aug=None,
          val_aug=None,
          test_augs=None,
          wav_len_val=5,
          wav_crop_len=30,
          img_height=256,
          img_weight=512,
          window_size=2048,
          sample_rate=32000,
          fmin=16,
          fmax=16386,
          power=2,
          mel_bins=256,
          hop_size=512,
          top_db=80.0,
          backbone='resnet34',
          pretrained=True,
          pretrained_weights=None,
          train=True,
          val=False,
          in_chans=1,
          alpha=1,
          eval_epochs=1,
          eval_train_epochs=1,
          warmup=0,
       

In [ ]:
# cfg = importlib.import_module('default_config')
# importlib.reload(cfg)
# cfg = importlib.import_module('cfg_ps_6_v2')
# importlib.reload(cfg)
# cfg = copy(cfg.cfg)

TEST_AUDIO_ROOT = "../input/birdclef-2023/test_soundscapes/"
cfg.val_data_folder = TEST_AUDIO_ROOT
cfg.pretrained = False


print(cfg.model, cfg.dataset, cfg.backbone, cfg.pretrained_weights, cfg.mel_norm)


RN34 base_ds resnet34 None False


In [ ]:
def batch_to_device(batch, device):
    batch_dict = {key: batch[key].to(device) for key in batch}
    return batch_dict

import librosa as lb
import librosa.display as lbd
import soundfile as sf
from  soundfile import SoundFile

class BirdDataset(Dataset):
    def __init__(self, data, sr=cfg.sample_rate, duration=5, step=None, resample=True, res_type="kaiser_fast"):

        self.data = data

        self.sr = sr

        self.duration = duration
        self.audio_length = self.duration * self.sr
        self.step = step or self.audio_length
        self.res_type = res_type
        self.resample = resample

    def __len__(self):
        return len(self.data)


    def read_file(self, filepath):
        audio, orig_sr = sf.read(filepath, dtype="float32")

        if self.resample and orig_sr != self.sr:
            audio = lb.resample(audio, orig_sr, self.sr, res_type=self.res_type)

        audios = []
        for i in range(self.audio_length, len(audio) + self.step, self.step):
            start = max(0, i - self.audio_length)
            end = start + self.audio_length
            audios.append(audio[start:end])

        if len(audios[-1]) < self.audio_length:
            audios = audios[:-1]

        images = [audio for audio in audios]
        images = np.stack(images)

        return images

    def __getitem__(self, idx):
        return {'input': self.read_file(self.data.loc[idx, "path"])}

In [ ]:
def gem(x, p=3, eps=1e-6):
    return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1.0 / p)


class GeM(nn.Module):
    # Generalized mean: https://arxiv.org/abs/1711.02512
    def __init__(self, p=3, eps=1e-6):
        super(GeM, self).__init__()
        self.p = Parameter(torch.ones(1) * p)
        self.eps = eps

    def forward(self, x):
        ret = gem(x, p=self.p, eps=self.eps)
        return ret

    def __repr__(self):
        return (self.__class__.__name__+ "(p="+ "{:.4f}".format(self.p.data.tolist()[0])+ ", eps="+ str(self.eps)+ ")")


class Mixup(nn.Module):
    def __init__(self, mix_beta):

        super(Mixup, self).__init__()
        self.beta_distribution = Beta(mix_beta, mix_beta)

    def forward(self, X, Y, weight=None):

        bs = X.shape[0]
        n_dims = len(X.shape)
        perm = torch.randperm(bs)
        coeffs = self.beta_distribution.rsample(torch.Size((bs,))).to(X.device)

        if n_dims == 2:
            X = coeffs.view(-1, 1) * X + (1 - coeffs.view(-1, 1)) * X[perm]
        elif n_dims == 3:
            X = coeffs.view(-1, 1, 1) * X + (1 - coeffs.view(-1, 1, 1)) * X[perm]
        else:
            X = coeffs.view(-1, 1, 1, 1) * X + (1 - coeffs.view(-1, 1, 1, 1)) * X[perm]

        Y = coeffs.view(-1, 1) * Y + (1 - coeffs.view(-1, 1)) * Y[perm]

        if weight is None:
            return X, Y
        else:
            weight = coeffs.view(-1) * weight + (1 - coeffs.view(-1)) * weight[perm]
            return X, Y, weight



class Net(nn.Module):
    def __init__(self, cfg):
        super(Net, self).__init__()

        self.cfg = cfg

        self.n_classes = cfg.n_classes

        self.mel_spec = ta.transforms.MelSpectrogram(
            sample_rate=cfg.sample_rate,
            n_fft=cfg.window_size,
            win_length=cfg.window_size,
            hop_length=cfg.hop_size,
            f_min=cfg.fmin,
            f_max=cfg.fmax,
            pad=0,
            n_mels=cfg.mel_bins,
            power=cfg.power,
            normalized=False,
        )

        self.amplitude_to_db = ta.transforms.AmplitudeToDB(top_db=cfg.top_db)
        self.wav2img = torch.nn.Sequential(self.mel_spec, self.amplitude_to_db)

        self.backbone = timm.create_model(
            cfg.backbone,
            pretrained=cfg.pretrained,
            num_classes=0,
            global_pool="",
            in_chans=cfg.in_chans,
        )

        if "efficientnet" in cfg.backbone:
            backbone_out = self.backbone.num_features
        else:
            backbone_out = self.backbone.feature_info[-1]["num_chs"]

        self.global_pool = GeM()

        self.head_23 = nn.Linear(backbone_out, self.n_classes)

        if cfg.pretrained_weights is not None:
            sd = torch.load(cfg.pretrained_weights, map_location="cpu")["model"]
            sd = {k.replace("module.", ""): v for k, v in sd.items()}
            self.load_state_dict(sd, strict=True)
            print("weights loaded from", cfg.pretrained_weights)
        self.loss_fn = nn.BCEWithLogitsLoss(reduction="none")

        self.mixup = Mixup(mix_beta=cfg.mix_beta)

        self.factor = int(cfg.wav_crop_len / 5.0)

    def forward(self, batch):


        x = batch["input"]
        bs, parts, time = x.shape
        x = x.reshape(parts, time)


        with autocast(enabled=False):
            x = self.wav2img(x)  # (bs, mel, time)
            if self.cfg.mel_norm:
                x = (x + 80) / 80

        x = x.permute(0, 2, 1)
        x = x[:, None, :, :]


        x = self.backbone(x)
        x = self.global_pool(x)
        x = x[:, :, 0, 0]
        logits = self.head_23(x)


        return logits.sigmoid()


In [ ]:
def get_state_dict(sd_fp):
    sd = torch.load(sd_fp, map_location="cpu")['model']
    sd = {k.replace("module.", ""):v for k,v in sd.items()}
    return sd

from scipy.stats.mstats import gmean

In [ ]:
df_test = pd.DataFrame(
     [(path.stem, *path.stem.split("_"), path) for path in Path(cfg.val_data_folder).glob("*.ogg")],
    columns = ["filename", "name" ,"id", "path"]
)
print(df_test.shape)
df_test.head()

(1, 4)


,filename,name,id,path
0,soundscape_29201,soundscape,29201,../input/birdclef-2023/test_soundscapes/sounds...


In [ ]:
N_CORES = 4
cfg.batch_size = 1

aug = None
test_ds =  BirdDataset(df_test, sr=cfg.sample_rate, duration=5, step=None, resample=True, res_type="kaiser_fast")
test_dl = DataLoader(test_ds, shuffle=False, batch_size = cfg.batch_size, num_workers = N_CORES)

test_ds[0]

{'input': array([[-0.00933968, -0.01086869, -0.01235762, ..., -0.01134745,
         -0.00982101, -0.01078924],
        [-0.01009817, -0.0092043 , -0.00916373, ..., -0.00932905,
         -0.0098142 , -0.01029071],
        [-0.01037158, -0.01044475, -0.01130228, ..., -0.00923787,
         -0.00863743, -0.00864352],
        ...,
        [-0.01015252, -0.01078276, -0.01009555, ..., -0.00809995,
         -0.00850733, -0.00849451],
        [-0.00842508, -0.00772012, -0.00932256, ..., -0.00986386,
         -0.01015088, -0.00889098],
        [-0.00817491, -0.00771294, -0.00703838, ..., -0.00935103,
         -0.00875765, -0.00919419]], dtype=float32)}

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else 'cpu'

In [ ]:
#state_dict = "../input/mel-gem-resnet-from-2021-2nd-place/first_model/checkpoint_last_seed123.pth"
#state_dict = "../input/mel-gem-resnet/first_model/checkpoint_last_seed123.pth"
state_dict = "/kaggle/input/birdclef-21-2nd-place-model-train-0-66/pre_model_multi_drop_v3/checkpoint_last_seed123_19.pth"
net = Net(cfg).eval().to(DEVICE)
sd = get_state_dict(state_dict)
print("loading dict")
net.load_state_dict(sd, strict=False)


loading dict


<All keys matched successfully>

In [ ]:
list(net.global_pool.parameters())

[Parameter containing:
 tensor([3.8073], requires_grad=True)]

In [ ]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [ ]:
with torch.no_grad():
    preds = []
    for batch in tqdm(test_dl):
        batch = batch_to_device(batch, DEVICE)
        with torch.cuda.amp.autocast():
            out = net(batch)
            preds += [out.cpu().numpy()]

  0%|          | 0/1 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/amp/autocast_mode.py:204: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
100%|██████████| 1/1 [00:17<00:00, 17.89s/it]


In [ ]:
preds

[array([[3.7735057e-04, 2.5086529e-03, 7.8755002e-05, ..., 2.1828107e-05,
         2.0724938e-04, 1.5816120e-04],
        [1.6118657e-02, 3.1912588e-02, 1.7036388e-02, ..., 1.0118068e-02,
         2.1254782e-02, 1.1181700e-02],
        [3.3139456e-02, 3.2493923e-02, 8.6593591e-03, ..., 2.9236428e-02,
         1.8872628e-02, 1.3146271e-02],
        ...,
        [1.1725673e-03, 4.3933834e-03, 1.7382255e-03, ..., 9.1173325e-04,
         2.1756338e-03, 1.1034964e-03],
        [1.3536820e-03, 4.2516449e-03, 2.8068305e-03, ..., 6.0286070e-04,
         1.7945615e-03, 3.1568757e-03],
        [8.5887574e-03, 1.7714016e-02, 1.4812327e-02, ..., 5.5276942e-03,
         2.1753570e-02, 9.8328078e-03]], dtype=float32)]

In [ ]:
df_train = pd.read_csv('../input/birdclef-2023/train_metadata.csv')

In [ ]:
filenames = df_test.filename.values.tolist()

bird_cols = list(pd.get_dummies(df_train['primary_label']).columns)
sub_df = pd.DataFrame(columns=['row_id']+bird_cols)

In [ ]:
sub_df

,row_id,abethr1,abhori1,abythr1,afbfly1,afdfly1,afecuc1,affeag1,afgfly1,afghor1,...,yebsto1,yeccan1,yefcan,yelbis1,yenspu1,yertin1,yesbar1,yespet1,yetgre1,yewgre1


In [ ]:
for i, file in enumerate(filenames):
    pred = preds[i]
    num_rows = len(pred)
    row_ids = [f'{file}_{(i+1)*5}' for i in range(num_rows)]
    df = pd.DataFrame(columns=['row_id']+bird_cols)

    df['row_id'] = row_ids
    df[bird_cols] = pred

    sub_df = pd.concat([sub_df,df]).reset_index(drop=True)


In [ ]:
sub_df

,row_id,abethr1,abhori1,abythr1,afbfly1,afdfly1,afecuc1,affeag1,afgfly1,afghor1,...,yebsto1,yeccan1,yefcan,yelbis1,yenspu1,yertin1,yesbar1,yespet1,yetgre1,yewgre1
0,soundscape_29201_5,0.000377,0.002509,0.000079,0.000187,0.000124,0.000123,0.019087,0.000153,0.004738,...,0.000029,0.000009,0.000903,0.000145,0.000359,0.000394,0.000169,0.000022,0.000207,0.000158
1,soundscape_29201_10,0.016119,0.031913,0.017036,0.023992,0.007365,0.006221,0.009199,0.017378,0.009449,...,0.002888,0.005010,0.041991,0.043452,0.017797,0.019962,0.010107,0.010118,0.021255,0.011182
2,soundscape_29201_15,0.033139,0.032494,0.008659,0.014866,0.002769,0.005024,0.003552,0.034560,0.011488,...,0.002442,0.007181,0.078498,0.018173,0.034065,0.015157,0.003684,0.029236,0.018873,0.013146
3,soundscape_29201_20,0.001623,0.004265,0.005578,0.004975,0.002719,0.001302,0.002801,0.003705,0.005192,...,0.000676,0.003257,0.006444,0.008011,0.002828,0.002939,0.002788,0.001481,0.003326,0.001991
4,soundscape_29201_25,0.002424,0.008998,0.007740,0.008605,0.002435,0.002442,0.004372,0.005116,0.011522,...,0.000821,0.004571,0.008730,0.010902,0.002888,0.007059,0.004195,0.002604,0.004504,0.003968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,soundscape_29201_580,0.001357,0.006344,0.002656,0.002384,0.000925,0.001293,0.000998,0.004070,0.002114,...,0.000170,0.002091,0.002120,0.004939,0.001750,0.003482,0.001618,0.001451,0.001200,0.000830
116,soundscape_29201_585,0.005084,0.005733,0.002340,0.002983,0.001940,0.002592,0.004465,0.005351,0.003759,...,0.000404,0.001355,0.004761,0.006357,0.003964,0.003928,0.004460,0.001214,0.002809,0.003406
117,soundscape_29201_590,0.001173,0.004393,0.001738,0.002602,0.001637,0.001178,0.001805,0.003596,0.002668,...,0.000420,0.001291,0.004835,0.006755,0.002938,0.002654,0.002991,0.000912,0.002176,0.001103
118,soundscape_29201_595,0.001354,0.004252,0.002807,0.002487,0.001598,0.001069,0.001525,0.004222,0.001657,...,0.000289,0.001397,0.003883,0.008972,0.001621,0.005613,0.004165,0.000603,0.001795,0.003157


In [ ]:
sub_df.to_csv('submission.csv',index=False)